In [ ]:
WORKER_COUNT = 8

In [1]:
# whether to log each feature and sequence status
verbose = True

In [2]:
import gc
import os
import pandas as pd
pd.options.display.max_rows = 5000
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [3]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [10]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s, [%(levelname)-8s] [%(filename)s:%(lineno)d] %(message)s',
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()

In [5]:
from deepvideoclassification.pretrained_CNNs import pretrained_model_names, pretrained_model_names_bucketed


Using TensorFlow backend.


In [6]:
from deepvideoclassification.pretrained_CNNs import test

# Create experiments lists and run experiments

* batch 1 = run frozen image MLP, LRCNNs and concat models on 1 of each pretrained_model_name in buckets (bucketed on feature sizes and limited to max sequence_length of 10)

* batch 2 = for best configurations from batch 1, run other pretrained models in buckets and run longer sequence lengths

* batch 3 = run trainable MLP and LRCNN on best performing frozen variants

* batch 4 = run trainable but initializing with best CNN weights

* batch 5 = run C3D models

* batch 6 = analyze effect of dropout and pooling with best model

# Batch 1

In [ ]:
experiment_batch_name = 'experiment_batch_1'

In [ ]:
# init list of experiments
experiments = []

In [ ]:
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
sequence_model_layer_counts = [1,2]

In [ ]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names_bucketed:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)

In [ ]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}

                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters
                        experiment['sequence_length'] = sequence_length

                        # add to list of experiments
                        experiments.append(experiment)

In [ ]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers
                                experiment['sequence_length'] = sequence_length

                                # add to list of experiments
                                experiments.append(experiment)

In [ ]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)
experiments['model_id'] = experiments.index

In [ ]:
experiments['WORKER'] = experiments['model_id'].apply(lambda x: x % WORKER_COUNT)

In [ ]:
experiments.to_csv(pwd + "experiments/" + experiment_batch_name + '.csv', index=False)

# Batch 2

In [ ]:
# for best configurations from batch 1, run other pretrained models 
# in buckets and run longer sequence lengths

# Batch 3

In [ ]:
# run trainable MLP and LRCNN on best performing frozen variants

In [ ]:
#######################
### image_MLP_trainable
#######################

architecture = 'video_LRCNN_trainable'

In [ ]:
#########################
### video_LRCNN_trainable
#########################

architecture = 'video_LRCNN_trainable'

# Batch 4

In [ ]:
# run trainable but initializing with best CNN weights

# Batch 5

In [ ]:
#######
### C3D
#######

architecture = 'C3D'

############
### C3Dsmall
############

architecture = 'C3Dsmall' 

# Analyze results

## load results.json for all models into dataframe

In [ ]:
path_models = pwd + 'models/'

results = []

for folder, subs, files in os.walk(path_models):
    for filename in files:
        if 'results.json' in filename:
            with open(os.path.abspath(os.path.join(folder, filename))) as f:
                data = json.load(f)
            results.append(data)

results = pd.DataFrame(results)        
results.sort_values("fit_val_acc", inplace=True, ascending=False)

In [ ]:
results.head(10).T

## Merge done status onto experiments

In [ ]:
experiments = pd.merge(experiments, results[['model_id','fit_val_acc']], left_on='model_id', right_on='model_id', how='left')
experiments['done'] = (experiments['fit_val_acc']>0).astype(int)
del experiments['fit_val_acc']

In [ ]:
experiments.head()

In [ ]:
print("{}/{} experiments done".format(experiments[experiments['done'] == 1].shape[0], len(experiments)))

## analyze remaining experiments, split on architecture

In [ ]:
experiments[experiments['done']==0]['architecture'].value_counts()

In [ ]:
experiments[experiments['architecture'] == 'video_MLP_concat']

In [ ]:
experiments[experiments['architecture'] == 'video_MLP_concat'].sort_values("sequence_length").head()

# Copy experiment files to s3

In [ ]:
model_ids = list(results['model_id'])
model_ids.sort()

In [ ]:
for i, model_id in enumerate(model_ids):
    
    path_model = pwd + '/models/' + str(model_id) + '/'

    # aws s3 ls on path returns 0 if it exists so check if doesn't exist, then sync
    if os.system("aws s3 ls s3://thesisvids/penguins/models/" + str(model_id) + "/") > 0:
        print("Synching {}/{} - model_id={}".format(i+1,len(model_ids),model_id))
        response = os.system("aws s3 sync " + path_model + " s3://thesisvids/penguins/models/" + str(model_id) + "/")
        if response != 0:
            print("ERROR syncing model_id = {}".format(model_id))
    else:
        print("Already synched {}/{} - model_id={}".format(i+1,len(model_ids),model_id))